In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

sections = {
    "최신뉴스": "https://news.nate.com/recent?mid=n0100",
    "정치": "https://news.nate.com/recent?mid=n0200",
    "경제": "https://news.nate.com/recent?mid=n0300",
    "사회": "https://news.nate.com/recent?mid=n0400",
    "세계": "https://news.nate.com/recent?mid=n0500",
    "IT/과학": "https://news.nate.com/recent?mid=n0600"
}

def scrape_nate_news():
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    for section, url in sections.items():
        print(f"\n### {section} ###\n")
        
        res = requests.get(url, headers=headers)
        res.encoding = 'utf-8'
        if not res.ok:
            print(f"Error Code = {res.status_code}")
            continue
        
        soup = BeautifulSoup(res.text, "html.parser")
        
        news_list = soup.select('div.mlt01 ul li')
        if not news_list:
            print("뉴스를 찾을 수 없습니다.")
            continue
        
        for idx, news in enumerate(news_list[:5], 1):
            title_tag = news.select_one('div.subject a')
            if not title_tag:
                continue
            
            title = title_tag.get_text(strip=True)
            link = urljoin(url, title_tag['href'])
            
            img_tag = news.select_one('img')
            img_url = None
            if img_tag and img_tag.get('src'):
                # src가 // 로 시작하면 https: 붙이기
                src = img_tag.get('src')
                img_url = urljoin('https:', src)
            
            print(f"{idx}. {title}")
            print(f"링크: {link}")
            if img_url:
                print(f"이미지: {img_url}")
                display(Image(url=img_url, width=200))
            else:
                print("이미지 없음")
            print("-" * 50)

# 실행
scrape_nate_news()



### 최신뉴스 ###

뉴스를 찾을 수 없습니다.

### 정치 ###

뉴스를 찾을 수 없습니다.

### 경제 ###

뉴스를 찾을 수 없습니다.

### 사회 ###

뉴스를 찾을 수 없습니다.

### 세계 ###

뉴스를 찾을 수 없습니다.

### IT/과학 ###

뉴스를 찾을 수 없습니다.


In [ ]:
import os
import requests
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    """
    네이버 웹툰의 특정 회차 이미지를 다운로드하는 함수
    :param title: 웹툰 제목
    :param no: 회차 번호
    :param url: 회차 URL
    """
    save_dir = os.path.join('img', title, str(no))
    os.makedirs(save_dir, exist_ok=True)
    print(f"'{save_dir}' 디렉토리에 이미지를 저장합니다.")

    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': url
        }
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        image_area = soup.select_one('div.wt_viewer')
        if not image_area:
            print("웹툰 이미지 영역을 찾을 수 없습니다.")
            return

        img_tags = image_area.find_all('img')
        
        for i, img_tag in enumerate(img_tags):
            if 'src' in img_tag.attrs:
                img_url = img_tag['src']
                
                print(f"{i+1}번째 이미지 다운로드 중...", end='\r')
                img_response = requests.get(img_url, headers=headers)
                img_response.raise_for_status()

                file_name = f"{i+1:03d}.jpg"
                file_path = os.path.join(save_dir, file_name)
                
                with open(file_path, 'wb') as f:
                    f.write(img_response.content)
        
        print(f"\n총 {len(img_tags)}개의 이미지를 성공적으로 다운로드했습니다.")

    except requests.exceptions.RequestException as e:
        print(f"HTTP 요청 오류 발생: {e}")
    except Exception as e:
        print(f"오류 발생: {e}")

webtoon_title = '일렉시드'
episode_no = 341
episode_url = 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed'

download_one_episode(webtoon_title, episode_no, episode_url)

result_value = f"download_one_episode('{webtoon_title}', {episode_no}, '{episode_url}')"
print(f"\n{result_value}")

'img\일렉시드\341' 디렉토리에 이미지를 저장합니다.
88번째 이미지 다운로드 중...
총 88개의 이미지를 성공적으로 다운로드했습니다.

download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')
